# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("OpenAI API KEY Exist")
else:
    print("Not exist")

MODEL = "gpt-4o-mini"
openai=OpenAI()

OpenAI API KEY Exist


In [3]:
system_message = """You are the customer support assistant for TechShop (an e-commerce site). Your name is Abigail.
Your Tasks:
Check order statuses
Initiate returns/exchanges
Provide product information
Track shipments
Answer general customer questions

Behavior Rules:
Always be friendly and helpful.
Address the customer by their name once you know it.
Show empathy when there is an issue: "I understand — I can imagine this is frustrating for you."
Order numbers must be 10 characters (e.g., ORD1234567).
Shipment tracking numbers must be 12 characters (e.g., KRG123456789012).
Returns must be requested within 14 days of delivery.
Customer satisfaction is your highest priority.

Communication Style:
Speak English; be polite but warm.
You may use emojis sparingly (only where appropriate).
Keep answers short and clear.
If needed, explain steps one-by-one.

Important Info:
Working hours: 09:00–22:00 (we are not 24/7).
Contact: destek@techshop.com or +90 850 123 45 67
Free shipping for orders over 500 TL.
Return period: 14 days from delivery.

If a request falls outside your capability, explain politely and offer an alternative.""" 

In [4]:
def chat(message,history):
    messages = [{"role" : "system" ,"content" : system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [18]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_order",
            "description" : "Fetches order status, shipment info and product details by order number",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "order_number": {
                        "type" :"string",
                        "description": "10-character order number (e.g., ORD1234567)",
                        "pattern" : "^[A-Z]{3}\\d{7}$",
                        "minLength": 10,
                        "maxLength":10
                    }
                },
                "required" :["order_number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name" : "initiate_return",
            "description" : "Starts a return process for the specified order",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_number": {
                        "type" : "string",
                        "description" : "10-character order number (e.g., ORD1234567)",
                        "pattern" : "^[A-Z]{3}\\d{7}$",
                        "minLength" :10,
                        "maxLength" :10
                    },
                    "reason": {
                        "type" : "string",
                        "description" : " Reason for return (e.g., 'damaged', 'wrong item', 'not satisfied')"
                        
                },
                "details":{
                    "type":"string",
                    "description" : "Detailed explanation (optional)"
            }
        },
                "required":["order_number","reason"]
            }
        }
    },
    {
        "type":"function",
        "function" : {
            "name" : "track_shipment",
      "description": "Shows shipment status and last known location using tracking number",
      "parameters": {
        "type": "object",
        "properties": {
          "tracking_number": {
            "type": "string",
            "description": "12-character shipment tracking number (e.g., KRG123456789)",
            "pattern": "^[A-Z]{3}\\d{9}$",
            "minLength": 12,
            "maxLength": 12
          }
        },
        "required": ["tracking_number"]
      }
    }
  },
    {
        "type": "function",
        "function": {
            "name" : "product_info",
            "description" : "Retrieces product details(price,stock,specs) by product name or code",
            "paramters": {
                "type" : "object",
                "properties": {
                    "product_query": {
                        "type" : "string",
                        "description": "Product name or code"
                    }
                },
                "required": ["product_query"]
            }
        }
    },
    {"type": "function",
     "function": {
      "name": "get_faq",
      "description": "Fetches FAQ entries from the database by topic",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "FAQ topic",
            "enum": ["shipping", "payment", "returns", "warranty", "membership"]
          }
        },
        "required": ["topic"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "create_support_ticket",
      "description": "Creates a support ticket for issues that can't be resolved immediately",
      "parameters": {
        "type": "object",
        "properties": {
          "full_name": {
            "type": "string",
            "description": "Customer full name"
          },
          "email": {
            "type": "string",
            "description": "Contact email (e.g., user@example.com)",
            "format": "email"
          },
          "subject": {
            "type": "string",
            "description": "Ticket subject"
          },
          "description": {
            "type": "string",
            "description": "Detailed description"
          }
        },
        "required": ["full_name", "email", "subject", "description"]
      }
    }
  }
]

In [21]:
import random
from datetime import datetime, timedelta

def check_order(order_number):
    """Fetches order information(simulated)."""
    statues = ["Preparing", "Shipped","Out for Delivery", "Delivered"]
    status = random.choice(statues)
    
    products = [
        {"name": "iPhone 15 Pro", "quantity": 1, "price": 45999},
        {"name": "AirPods Pro", "quantity": 1, "price": 8999},
        {"name": "Apple Watch Series 9", "quantity": 1, "price": 15999}
    ]
    
    product = random.choice(products)

    return {
    "success": True,
    "order_number" : order_number,
    "date": (datetime.now() - timedelta(days=random.randint(1,10))).strftime("%Y-%m-%d"),
    "status" : status,
    "products" : [product],
    "total" : product["price"],
    "shipping_provider" : random.choice(["Aras Kargo", "Yurtici Kargo", "Mng Kargo"]),
    # tracking_number format: 3 letters + 9 digits = 12 chars (matches tools pattern)
    "tracking_number": f"KRG{random.randint(100000000, 999999999)}",
    "estimated_delivery": (datetime.now() + timedelta(days=random.randint(1, 3))).strftime("%Y-%m-%d")
    }

In [23]:
def initiate_return(order_number,reason,details=""):
    """Starts a return process (simulated)."""

    return_id = f"RET{random.randint(100000,999999)}"

    return {
    "success" :True,
    "return_id" : return_id,
    "order_number" : order_number,
    "reason": reason,
    "status" : "Return Request Received",
    "message" : "Your return request has been created successfully.The courier will pick up the item within 2 business days.",
    "return_code": return_id,
    "shipping_fee": "Free if issue caused by us",
    "refund_time": "5-7 business days after the item arrives at the warehouse"
    }

SyntaxError: incomplete input (914743823.py, line 6)

In [ ]:
def
